In [1]:
!sudo apt -y update
!sudo apt install -y wget curl unzip
!wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
!dpkg -i libu2f-udev_1.1.4-1_all.deb
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
!wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
!unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
!chmod +x /tmp/chromedriver
!mv /tmp/chromedriver /usr/bin/chromedriver
!pip install selenium
!apt-get update -qq
!apt-get install -y fonts-nanum*
!apt-get install fonts-nanum* -qq

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [634 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,325 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.c

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions

from bs4 import BeautifulSoup
import re

import sys
if 'ipykernel' in sys.modules:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

import pandas as pd
import time

In [3]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")
user_agent = "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0"
options.add_argument('user-agent=' + user_agent)

driver = webdriver.Chrome(options=options)

### 교보문고

In [ ]:
import pickle
with open('df_kyob.pkl', 'rb') as f: # 교보문고 파일 가져오기
    kyob = pickle.load(f)
kyob # 데이터프레임 형식


,순위,판매상품ID,상품코드,상품명,저자,출판사,분야,발행(출시)일자,정가,판매가,할인율,적립율,적립예정포인트
0,1,S000000401269,9788901047546,한 권으로 읽는 조선왕조실록,박영규,웅진지식하우스,역사/문화,2013-01-13,18000,16200,10,5,900
1,4,S000000401458,9788901052922,괜찮아,최숙희,웅진주니어,유아(0~7세),2009-04-22,10000,9000,10,5,500
2,5,S000000401766,9788901058405,철학 콘서트,황광우,웅진지식하우스,인문,2006-06-28,15000,13500,10,5,750
3,6,S000000403626,9788901090184,클루지,개리 마커스,갤리온,인문,2008-11-24,13800,12420,10,5,690
4,8,S000000403841,9788901093154,넛지 : 똑똑한 선택을 이끄는 힘,리처드 탈러 외,리더스북,경제/경영,2009-04-22,15500,13950,10,5,770
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1462,1993,S000061585932,9791191891201,잘될 수밖에 없는 너에게,최서영,북로망스,시/에세이,2022-08-18,16000,14400,10,5,800
1463,1994,S000061671940,9791192300245,마흔에 읽는 니체,장재형,유노북스,인문,2022-09-01,16000,14400,10,5,800
1464,1995,S000061694154,9788936438838,아버지의 해방일지,정지아,창비,소설,2022-09-02,15000,13500,10,5,750
1465,1996,S000061863194,9788950941819,세상에서 가장 쉬운 본질육아,지나영,21세기북스,가정/육아,2022-09-28,18800,16920,10,5,940


In [ ]:
book_n = kyob['판매상품ID'] # 판매상품 ID로 책 검색

In [ ]:
from tqdm import tqdm
book_info = []

for n in tqdm(kyob['판매상품ID']) : # 데이터프레임 판매상품ID 열을 순서대로 갸져온다
    url = f'https://product.kyobobook.co.kr/detail/{n}' # 책 페이지
    driver.get(url)

    time.sleep(3)
    try :
        genre = driver.find_element(By.CLASS_NAME, 'category_list_item').text # 책 장르
        contents = driver.find_element(By.CLASS_NAME, 'intro_bottom').text # 책 소개
        index = driver.find_element(By.CLASS_NAME, 'book_contents_item').text # 책 목차
    except :
        book_info.append([n]) # 에러가 뜰 경우 nan값으로 채운다
        continue
    book_info.append([n, genre, contents, index]) # 순서대로 ID, 장르, 소개, 목차 가져온다


100%|██████████| 721/721 [1:57:40<00:00,  9.79s/it]


In [ ]:
book_df = pd.DataFrame(book_info) # 데이터들을 데이터프레임 형식으로 변경한다
book_df

,0,1
0,S000001302195,https://contents.kyobobook.co.kr/sih/fit-in/19...
1,S000001302216,https://contents.kyobobook.co.kr/sih/fit-in/19...
2,S000001302223,https://contents.kyobobook.co.kr/sih/fit-in/19...
3,S000001306810,https://contents.kyobobook.co.kr/sih/fit-in/19...
4,S000001306889,https://contents.kyobobook.co.kr/sih/fit-in/19...
...,...,...
716,S000061585932,https://contents.kyobobook.co.kr/sih/fit-in/19...
717,S000061671940,https://contents.kyobobook.co.kr/sih/fit-in/19...
718,S000061694154,https://contents.kyobobook.co.kr/sih/fit-in/19...
719,S000061863194,https://contents.kyobobook.co.kr/sih/fit-in/19...


In [ ]:
book_df.to_pickle('book_data_final.pkl')